## 1. Environment Setup
Install and import all required Python packages, including SymPy, PyTorch, CVXPY, and supporting libraries for optimization and preprocessing.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
# Uninstall all existing versions of sympy
!pip uninstall sympy -y

# Install sympy version 1.12
!pip install sympy==1.12

# Import sympy and check the installed version
import sympy
print("Sympy version:", sympy.__version__)

In [ ]:
# Install cvxpylayers for differentiable convex optimization layers
!pip install cvxpylayers

# Install cvxpy for convex optimization modeling
!pip install cvxpy

## 2. Data Loading
Read the appointment scheduling dataset from a CSV file. The data contains scheduled visit times and other features for patient appointments.


In [ ]:
file_path3 = '/content/scheduled_visits_Bita.csv'
df = pd.read_csv(file_path3)

## 3. Feature Engineering
Extract useful date and time components from the `scheduled_time` column. Also create new features like time of day, day of week, and whether the appointment falls on a weekend.


In [ ]:
df['scheduled_time'] = pd.to_datetime(df['scheduled_time'])

# Time components
df['hour'] = df['scheduled_time'].dt.hour
df['minute'] = df['scheduled_time'].dt.minute

# Date components
df['day'] = df['scheduled_time'].dt.day
df['month'] = df['scheduled_time'].dt.month
df['year'] = df['scheduled_time'].dt.year
df['week_of_year'] = df['scheduled_time'].dt.isocalendar().week.astype(int)
df['day_of_year'] = df['scheduled_time'].dt.dayofyear

# Day of week (0 = Monday, 6 = Sunday)
df['day_of_week'] = df['scheduled_time'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Time of day bins
df['time_of_day'] = pd.cut(
    df['hour'],
    bins=[-1, 6, 12, 17, 21, 24],
    labels=['night', 'morning', 'afternoon', 'evening', 'late night']
)


In [ ]:
# Label encode categorical features
categorical_cols = ['floor_id', 'department', 'diagnosis', 'appointment_type', 'link_flag', 'time_of_day']
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

## 4. Subset Selection
Focus on a subset of data where the event type is 'Exam' and further filter by month (e.g., January). Select the features and target variable (`duration_min`) to be used in modeling.


In [ ]:
df_exam = df[df['event_desc'] == 'Exam']
df_exam_sub = df_exam[df_exam['month'] == 1]

In [ ]:
selected_columns = [
    # 'time_of_day', 'day_of_week', 'is_weekend', 'day', 'month',
    'floor_id', 'department',
    # 'diagnosis',
    'appointment_type', 'link_flag',
    'scheduled_duration_min',
    'duration_min',
]

df_exam_sub_cols = df_exam_sub[selected_columns]

In [ ]:
# Drop rows where duration_min is missing
df_NN = df_exam_sub_cols.dropna(subset=['duration_min'])

# Select features and target variable
X = df_NN[['appointment_type', 'department', 'floor_id']].values
y = df_NN['duration_min'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Get the number of input features
input_dim = X_train.shape[1]

# Normalize target values to improve training stability
y_train_max = y_train.max()
y_train = y_train / y_train_max
y_test = y_test / y_train_max

## 5. SCT Evaluation Functions
Define helper functions to compute the sum of completion times (SCT) given a task ordering, and to measure how closely a model's predictions match the optimal ordering.


In [ ]:
# -------------------------------
# SCT Evaluation
# -------------------------------

def calculate_sct_ordering(durations, ordering):
    # Compute SCT for given ordering of durations
    sorted_durations = durations[ordering]
    return np.sum(np.cumsum(sorted_durations))


def calculate_relative_sct_error(model, X_test, y_test):
    # Compute predicted durations and compare SCT error with optimal ordering
    model.eval()
    with torch.no_grad():
        y_pred = model(torch.tensor(X_test, dtype=torch.float32)).squeeze().numpy()
    pred_order = np.argsort(y_pred)
    true_order = np.argsort(y_test)
    pred_sct = calculate_sct_ordering(y_test, pred_order)
    true_sct = calculate_sct_ordering(y_test, true_order)
    return (pred_sct - true_sct) / (true_sct + 1e-8)

## 6. Model Definition
Define a simple 3-layer neural network using PyTorch. The output uses a Softplus activation to ensure non-negative predictions (valid durations).


In [ ]:
# -------------------------------
# Neural Network Model
# -------------------------------
class SmallNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        # Simple 3-layer feedforward network with Softplus output
        self.model = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Softplus()  # Ensures non-negative output
        )

    def forward(self, x):
        return self.model(x)

## 7. Custom Loss Functions
Implement loss functions for training. These are used to compare the effectiveness of training.


In [ ]:
# -------------------------------
# Expected Regret Loss using sigmoid-based misordering probability
# -------------------------------
def expected_regret_loss_sigmoid(y_true, y_pred, num_pairs=200000):
    N = y_true.size(0)

    # Randomly sample pairs of indices
    idx = torch.randint(0, N, (num_pairs, 2))
    i, j = idx[:, 0], idx[:, 1]

    # Compute prediction difference for each pair
    diff_pred = y_pred[i] - y_pred[j]

    # Estimate probability that the pair is misordered (i ranked above j incorrectly)
    misorder_prob = torch.sigmoid(diff_pred / 0.1)

    # Compute regret only when the true order is violated
    regret = torch.relu((y_true[j] - y_true[i]) * misorder_prob)

    # Return the average regret over all sampled pairs
    return torch.mean(regret)

# -------------------------------
# Rank loss based on pairwise comparisons and sigmoid scoring
# -------------------------------
def rank_loss(y_true, y_pred, num_pairs=200000):
    N = y_true.size(0)

    # Randomly sample pairs of indices
    idx = torch.randint(0, N, (num_pairs, 2))
    i, j = idx[:, 0], idx[:, 1]

    # Keep only pairs where true label of i < j
    mask = (y_true[i] < y_true[j])
    i_masked = i[mask]
    j_masked = j[mask]

    # Return zero if no valid pairs are found
    if i_masked.numel() == 0:
        return torch.tensor(0.0, requires_grad=True)

    # Compute pairwise differences in predictions
    diff_pred = y_pred[j_masked] - y_pred[i_masked]

    # Apply negative log-sigmoid loss
    log_s = torch.log(torch.sigmoid(diff_pred) + 1e-12)
    return -torch.mean(log_s)

# -------------------------------
# Mean Squared Error loss (standard regression loss)
# -------------------------------
def mse_loss(y_true, y_pred, num_pairs=None):  # num_pairs unused, just for API compatibility
    return torch.mean((y_true - y_pred) ** 2)

In [ ]:
# -------------------------------
# SPO+ Loss via CVXPY Layer
# -------------------------------
def create_spo_plus_layer(n_jobs):
    # Define and return the SPO+ optimization layer
    x = cp.Variable(n_jobs)
    c = cp.Parameter(n_jobs)
    problem = cp.Problem(cp.Minimize(c @ x), [x >= 0, cp.sum(x) == n_jobs])
    return CvxpyLayer(problem, parameters=[c], variables=[x])

def spo_plus_loss_true(y_true, y_pred, spo_layer):
    # Calculate SPO+ surrogate loss minus the true SPO loss
    n = y_true.size(0)
    device = y_true.device
    true_order = torch.argsort(y_true)
    true_completion_times = torch.arange(1, n + 1, dtype=torch.float32).to(device)
    spo_true_loss = torch.sum(y_true[true_order] * true_completion_times)

    x_opt, = spo_layer(y_pred)
    surrogate = 2 * torch.sum(y_pred * x_opt) - torch.sum(y_true * x_opt)

    return surrogate - spo_true_loss

## 8. Training Loop
Train a model using any loss function and evaluate it using SCT-based metrics. Early stopping is applied if the loss does not improve.


In [ ]:
# -------------------------------
# Training loop for a model using a custom loss function
# -------------------------------

def train_model(model, optimizer, loss_fn, X_train, y_train, X_test, y_test,
                num_iterations=1000, num_pairs=200000, eval_interval=100,
                patience=200, min_delta=1e-5):

    model.train()
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

    best_loss = float("inf")
    no_improvement_counter = 0
    sct_diffs = []
    steps = []

    for step in range(num_iterations):
        # Forward + backward pass
        optimizer.zero_grad()
        y_pred_train = model(X_train_tensor).squeeze()
        train_loss = loss_fn(y_train_tensor, y_pred_train, num_pairs)
        train_loss.backward()
        optimizer.step()

        # Evaluate model every eval_interval steps (or last step)
        if step % eval_interval == 0 or step == num_iterations - 1:
            sct_diff = calculate_relative_sct_error(
                model, X_test, y_test,
                model_type="Real Model", print_results=False
            )
            print(f"Step {step:4d} | Train Loss: {train_loss.item():.4f} | sct_error: {sct_diff:.2f}")
            sct_diffs.append(sct_diff)
            steps.append(step)

        # Early stopping if no significant improvement
        if best_loss - train_loss.item() > min_delta:
            best_loss = train_loss.item()
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1
            if no_improvement_counter >= patience:
                print(f"Early stopping at step {step}, best train loss: {best_loss:.6f}")
                break

    # Final evaluation: compute test MSE
    model.eval()
    with torch.no_grad():
        y_pred_test = model(torch.tensor(X_test, dtype=torch.float32)).squeeze().numpy()
    test_mse = mean_squared_error(y_test, y_pred_test)
    print(f"Final Test MSE: {test_mse:.4f}")

    return steps, sct_diffs

In [ ]:
# -------------------------------
# Training Loop (Mini-batch SPO+)
# -------------------------------
def train_model_spo(model, optimizer, X_train, y_train, X_test, y_test,
                    num_iterations=1000, batch_size=1000,
                    eval_interval=100, min_delta=1e-9, patience=1000):
    model.train()
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).squeeze()

    best_loss = float('inf')
    steps, rel_sct_errors = [], []
    no_improvement_counter = 0
    last_logged_step = -1

    for step in range(num_iterations):
        # Sample a mini-batch
        idx = torch.randperm(len(X_train_tensor))[:batch_size]
        X_batch = X_train_tensor[idx]
        y_batch = y_train_tensor[idx]

        # Create new CVXPY layer for this batch size
        spo_layer = create_spo_plus_layer(batch_size)

        # Forward + backward pass
        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = spo_plus_loss_true(y_batch, y_pred, spo_layer)
        loss.backward()
        optimizer.step()

        # Evaluation step
        if step % eval_interval == 0:
            rel_error = calculate_relative_sct_error(model, X_test, y_test)
            print(f"Step {step:4d} | SPO+ Loss: {loss.item():.4f} | Rel. SCT Error: {rel_error:.4f}")
            steps.append(step)
            rel_sct_errors.append(rel_error)
            last_logged_step = step

        # Check for improvement
        if best_loss - loss.item() > min_delta:
            best_loss = loss.item()
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1
            if no_improvement_counter >= patience:
                print(f"Early stopping at step {step}")
                break

    # Final evaluation if last step wasn't logged
    if last_logged_step != step:
        rel_error = calculate_relative_sct_error(model, X_test, y_test)
        print(f"Final Step {step:4d} | SPO+ Loss: {loss.item():.4f} | Rel. SCT Error: {rel_error:.4f}")
        steps.append(step)
        rel_sct_errors.append(rel_error)

    return steps, rel_sct_errors

## 9. Comparison of All Loss Functions
Plot relative SCT error curves for all trained models. Final performance is highlighted and extended to the end of training for visual comparison.


In [ ]:
# Get input feature size
input_dim = X_train.shape[1]

results = {}

# Train and evaluate the model using different loss functions
for loss_name, loss_fn in {
    'Regret Loss (Sigmoid)': expected_regret_loss_sigmoid,
    'Rank Loss': rank_loss,
    'MSE Loss': mse_loss
}.items():
    print(f"\nTraining with {loss_name}")

    # Initialize a new model and optimizer for each loss function
    model = SmallNN(input_size=input_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

    # Train the model and collect evaluation metrics
    steps, sct_diffs = train_model(
        model, optimizer, loss_fn,
        X_train, y_train, X_test, y_test,
        num_iterations=2000, eval_interval=50
    )

    # Store the results
    results[loss_name] = (steps, sct_diffs)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
steps, rel_errors = train_model_spo(model, optimizer, X_train, y_train, X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Assign fixed colors to each loss function
colors = {
    'Regret Loss (Sigmoid)': 'blue',
    'Rank Loss': 'orange',
    'MSE Loss': 'green',
    'SPO+': 'red'
}

# Combine all results, including SPO+ results from earlier
all_results = results.copy()
all_results['SPO+'] = (steps, rel_errors)

# Define maximum x-value for horizontal dashed line extension
max_step = 2000

# Plot each model's training curve
for loss_name, (loss_steps, loss_errors) in all_results.items():
    color = colors.get(loss_name, None)

    # Plot the training curve
    plt.plot(loss_steps, loss_errors, marker='o', label=loss_name, color=color)

    # Mark and annotate the final point
    final_step = loss_steps[-1]
    final_error = loss_errors[-1]
    plt.scatter(final_step, final_error, color=color)
    plt.text(final_step, final_error, f"{final_error:.3f}", fontsize=8,
             ha='right', va='bottom', color=color)

    # Extend a dashed line to the right to compare final performance
    if final_step < max_step:
        plt.hlines(y=final_error, xmin=final_step, xmax=max_step,
                   linestyles='dashed', colors=color, alpha=0.5)

# Add labels, title, legend, and formatting
plt.xlabel("Training Step")
plt.ylabel("Relative SCT Error")
plt.title("Comparison of Loss Functions")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()